In [1]:
from main import plot_curs_dist, plot_curs_time, plot_cur_runtime, leverage_cur, subspace_cur
import pandas as pd
import numpy as np
from sklearn import datasets

Genetics data reference:
Molecular Classification of Cancer: Class Discovery and Class Prediction by Gene Expression

Science 286:531-537. (1999). Published: 1999.10.14

T.R. Golub, D.K. Slonim, P. Tamayo, C. Huard, M. Gaasenbeek, J.P. Mesirov, H. Coller, M. Loh, J.R. Downing, M.A. Caligiuri, C.D. Bloomfield, and E.S. Lander

Used from Kaggle.

The genetics dataset used contained both numeric variables of gene activation and a categorical variable indicating whether researchers believed the activation indicated the gene was active. Since the use of PCA on categorical variables potentially has some issues, these were dropped leaving only the activations. The columns correspond to patients and the rows correspond to gene activation levels. Both the other datasets are pulled from sklearn and are used without any modification.

In [5]:
def get_gene_data()->pd.DataFrame:
    '''Gets the genetics dataset and removes all the categorical variables
    before returning it as a pandas dataframe.
    '''
    gene_data = pd.read_csv("./genes.csv")
    def is_number(x):
        try:
            float(x)
            return True
        except:
            return False
    is_numeric = np.vectorize(is_number, otypes=[bool])
    return gene_data.loc[:,is_numeric(gene_data.columns)]
    
def O_L2(dataset:pd.DataFrame):
    '''Calculates the largest outlier distance of the 
    L2 norm of the column from the average L2 norm of the columns.'''
    centered = dataset - dataset.mean(axis=0)
    norm = np.linalg.norm(centered, axis=0)
    return max(norm) - np.average(norm)

Main run of data used in the paper (section 5). Makes plots of the comparison of the two algorithms and gives the O_L2 measure described in the document. #note figures in paper need to be updated to reflect the mean/sd of the leverage data as used here. 

In [6]:
boston = pd.DataFrame(datasets.load_boston(return_X_y=True)[0])
boston = boston - boston.mean(axis=0)
plot_curs_dist(boston, 13, 10, 15, "./Figs/boston/", data_name="Boston")
print("finished dist plots for boston")
plot_curs_time(boston, 13, 10, 15, "./Figs/boston/", data_name="Boston")
print("finished time plot for boston")
plot_cur_runtime(boston, 13, 10, 15, 15, "./Figs/boston/", data_name="Boston")
print("finished runtime plot for boston")
print(f"Boston O_L2: {O_L2(boston)}")

wine = pd.DataFrame(datasets.load_wine(return_X_y=True)[0])
wine = wine - wine.mean(axis=0)
plot_curs_dist(wine, 13, 10, 15, "./Figs/wine/", data_name="Wine")
print("finished dist plots for wine")
plot_curs_time(wine, 13, 10, 35, "./Figs/wine/", data_name="Wine")
print("finished time plot for wine")
plot_cur_runtime(wine, 13, 10, 15, 15, "./Figs/wine/", data_name="Wine")
print("finished runtime plot for wine")
print(f"Wine O_L2: {O_L2(wine)}")

genetics = get_gene_data()
genetics = genetics - genetics.mean(axis=0)
plot_curs_dist(genetics, 13, 10, 15, "./Figs/genetics/", data_name="Genes")
print("finished dist plots for genetics")
plot_curs_time(genetics, 13, 10, 15, "./Figs/genetics/", data_name="Genes")
print("finished time plot for genetics")
plot_cur_runtime(genetics, 13, 10, 15, 15, "./Figs/genetics/", data_name="Genes")
print("finished runtime plot for genetics")
print(f"Gene O_L2: {O_L2(boston)}")

1
2
3
4
5
6
7
8
9
10
11
12
13
finished dist plots for boston
1
2
3
4
5
6
7
8
9
10
11
12
13
finished time plot for boston
1
2
3
4
5
6
7
8
9
10
11
12
13
finished runtime plot for boston
Boston O_L2: 3185.9119100724215
1
2
3
4
5
6
7
8
9
10
11
12
13
finished dist plots for wine
1
2
3
4
5
6
7
8
9
10
11
12
13
finished time plot for wine
1
2
3
4
5
6
7
8
9
10
11
12
13
finished runtime plot for wine
Wine O_L2: 3841.298673694798
1
2
3
4
5
6
7
8
9
10
11
12
13
finished dist plots for genetics
1
2
3
4
5
6
7
8
9
10
11
12
13
finished time plot for genetics
1


Code used to generate the matrix example showing a potential issue with using the leverage sampling algorithm (section 3).

In [ ]:
matrix = np.array(  [[-3,-6.33,-.106],
                    [0,4.67, -.65],
                    [3,1.66, .75 ]])
matrix = matrix - np.mean(matrix, axis=0)
U,Sigma,V_t = np.linalg.svd(matrix)
print(f"Dist: {1 - (np.dot(U[:,1], matrix)/np.linalg.norm(matrix, axis=0))**2}")
def leverage(V,k,j):
        return np.dot(V[j,:k].T,V[j,:k])
V = V_t.T
k =1
col_leverage = np.array([leverage(V,k,j)/k for j in range(matrix.shape[1])])
print(f"Leverage: {col_leverage}")